# Trabalho Final

**Aluno**: Carlos Alberto Rocha Cardoso

**Matrícula**: 90079


## 03 - Tratamento da base *formacao.csv*

Essa base descreve a formação acadêmica dos funcionários, representados pela coluna chave **perfil_id**, apresentando o ano de início e término da formação (coluna **date**), o título obtido (coluna **degree**), a área de estudo (**major**) e a escola (**school**). Podem existir mais de um registro por **perfil_id**, no caso do funcionário possuir formações diversas.

Todos os 111 funcionários constam nessa base.

In [522]:
import numpy as np
import pandas as pd
#import warnings

#warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#Aumentar a quantidade de caracteres exibidos para uma coluna (originalmente 50)
pd.options.display.max_colwidth = 100

formacao = pd.read_csv("bases/formacao.csv")

# Análise inicial da qualidade dessa base
print("Quantidade de registros: ", len(formacao))
print("Quantidade de valores distintos de perfil_id: ", formacao.perfil_id.nunique())
print("Dados faltantes em degree: ", len(formacao.loc[formacao.degree.isnull()]))
print("Dados faltantes em date: ", len(formacao.loc[formacao.date.isnull()]))
print("Dados faltantes em major: ", len(formacao.loc[formacao.major.isnull()]))
print("Dados faltantes em school: ", len(formacao.loc[formacao.school.isnull()]))
print("Dados faltantes em degree + major: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull())]))
print("Dados faltantes em degree + major + school: ", len(formacao.loc[(formacao.degree.isnull()) & (formacao.major.isnull()) & (formacao.school.isnull())]))
print("Valores distintos em degree: ", len(formacao.groupby('degree').nunique()))
print("Valores distintos em major: ", len(formacao.groupby('major').nunique()))
print("Valores distintos em school: ", len(formacao.groupby('school').nunique()))


Quantidade de registros:  220
Quantidade de valores distintos de perfil_id:  111
Dados faltantes em degree:  29
Dados faltantes em date:  7
Dados faltantes em major:  50
Dados faltantes em school:  1
Dados faltantes em degree + major:  13
Dados faltantes em degree + major + school:  1
Valores distintos em degree:  135
Valores distintos em major:  114
Valores distintos em school:  122


In [523]:
formacao.head(5)

,date,degree,major,school,perfil_id
0,2016 – 2016,"Nome do diploma\r\nExecutive Education, Innovation & Entrepreneurship",NaN,Stanford University,0
1,2007 – 2010,Nome do diploma\r\nEconomics,Área de estudo\r\nEconomics,Universidade Federal de Minas Gerais,0
2,2015 – 2015,Nome do diploma\r\nMaster of Business Administration (MBA),Área de estudo\r\nGestão de Negócios,Centro Universitário Newton Paiva,1
3,2009 – 2012,Nome do diploma\r\nBacharel em Design de Ambientes,Área de estudo\r\nDesign de Interiores,Universidade do Estado de Minas Gerais,1
4,2016 – 2019,Nome do diploma\r\nBachelor’s Degree,Área de estudo\r\nComputer Technology/Computer Systems Technology,Universidade Federal de Minas Gerais,2


## 03.1 - Tratamento das datas

Um primeiro tratamento feito será o de data. A coluna **date** que indica o íncio e o fim do curso e será quebrada em duas colunas numéricas (**inicio** e **fim**), e ainda numa terceira coluna indicando a duração estimada do curso em anos baseado na informação obtida (**duracao**).

In [524]:
inicio_termino = formacao.date.str.split('–')

formacao['inicio'] = pd.to_numeric(inicio_termino.str.get(0))
formacao['termino'] = pd.to_numeric(inicio_termino.str.get(1))
formacao['duracao'] = (formacao['termino'] - formacao['inicio']) + 1


## 03.2 - Tratamento da coluna de grau de escolaridade

Uma vez que a coluna **degree** não é muito precisa, uma coluna **grau** será criada. Ela começará vazia e será armazenado a inferência do grau de formação baseado na análise das demais colunas.

Uma primeira estratégia será avaliar o nome do **degree** para inferir se ele contém algum fragmento que possa indicar algum dos seguintes graus de escolaridade, representados por números: **Ensino Médio**=1, **Técnico**=2, **Superior**=3 ou **Pós-Graduação**=4.

Para isso, utilize a função [`contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html).

In [532]:
formacao['grau'] = None
formacao['desc_grau'] = formacao.degree
formacao['instituicao'] = formacao.school
formacao['area'] = formacao.major

# Troca os valores não informados de proficiência por "Não informado"
formacao.instituicao.fillna("-", inplace=True)
formacao.desc_grau.fillna("-", inplace=True)
formacao.area.fillna("-", inplace=True)

formacao.desc_grau = formacao.desc_grau.str.replace("Nome do diploma\r\n", "")

formacao.loc[formacao.desc_grau.str.contains("Non-degree Student", case=False), "grau"] = 1
formacao.loc[formacao.desc_grau.str.contains("Ensino Médio$", case=False), "grau"] = 1
formacao.loc[formacao.desc_grau.str.contains("High School$", case=False), "grau"] = 1
formacao.loc[formacao.desc_grau.str.contains("^Cur.*Admi", case=False), "grau"] = 1
formacao.loc[formacao.desc_grau.str.contains("^Under", case=False), "grau"] = 1

formacao.loc[formacao.desc_grau.str.contains("T[é,e]ch?ni", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("Google", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("Help Desk", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("Turismo", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("English Program", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("Minor", case=False), "grau"] = 2
formacao.loc[formacao.desc_grau.str.contains("Gastronomia", case=False), "grau"] = 2

formacao.loc[formacao.desc_grau.str.contains("T[é,e]ch?n[ó,o]", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("Bach|B\.S\.", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("^Eng.*Prod|^Eng.*Comp", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("Anali.*Sist", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("^(Curs.*)?Admin", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("^Ciência", case=False, regex=True), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("^Psic[ó,o]l", case=False), "grau"] = 3
formacao.loc[formacao.desc_grau.str.contains("^Grad", case=False), "grau"] = 3

formacao.loc[formacao.desc_grau.str.contains("(M[ae]st|MBA)", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("P[ó,o]s", case=False, regex=True), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Executive", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Inter(niship|change)", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Market", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Gestão de vendas e Negociação", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Processos Gerenciais", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("Study Abroad", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains(".*Especialista$|.*Especialização$", case=False), "grau"] = 4
formacao.loc[formacao.desc_grau.str.contains("[\s]Speciali|^Speci", case=False), "grau"] = 4


#formacao.loc[formacao.degree.str.contains("[\s]Speciali|^Speci", case=False)]
#formacao.loc[formacao.degree.str.contains("Inter(niship|change)", case=False)]


C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


Para os demais casos do grau de escolaridade não cobertos pela análise do texto, deverá ser feita uma análise mais profunda. Tente encontrar outros padrões para definir o grau de escolaridade, por exemplo, o nome da instituição de ensino (campo **school**) ou, então, o tempo de duração.

In [526]:
formacao.loc[formacao.instituicao.str.contains("^Col[e,é]gio", case=False) 
             & (formacao.duracao > 5), "grau"] = 1

formacao.loc[formacao.instituicao.str.contains("Caldwell.*Technical", case=False), "grau"] = 2

formacao.loc[(formacao.duracao > 2)
             & (formacao.grau.isnull())
             & (formacao.instituicao.str.contains("Uni|Fed", case=False)), 'grau'] = 3

formacao.loc[formacao.desc_grau.str.contains("2011", case=False), "grau"] = 3

formacao.loc[formacao.instituicao.str.contains("Ecomm", case=False), "grau"] = 4
formacao.loc[formacao.area.str.contains("Softwaretechnik|Inno", case=False), "grau"] = 4

#formacao[['perfil_id','date','grau','desc_grau','major','school']].sort_values(['grau','desc_grau'])
#formacao.loc[formacao.grau.isnull()]


## 03.3 - Tratamento da coluna de grau de escolaridade

A coluna **school** apresenta uma vasta de gama de categorias. Porém numa análise visual dos seus diferentes valores é possível ver que algumas instituições de ensino apresentam variações de nome. Foi feita uma substituição para agrupar esses casos num único nome. O resultao do tratamento é armazenado na nova coluna **instituicao**.

In [527]:
formacao.instituicao = formacao.school.str.replace("^Univ.*.Fed.*.M.*.G.*", "UFMG - Universidade Federal de Minas Gerais") \
                        .str.replace("^Fun.*Get.*as", "FGV - Fundação Getúlio Vargas") \
                        .str.replace("(^Uni-BH|^Cent.*Uni.*zonte.*)", "UNI-BH - Centro Universitário de Belo Horizonte") \
                        .str.replace(".*Uni.*UNA.*", "Centro Universitário UNA") \
                        .str.replace(".*Col.*Santo.*nio", "CSA - Colégio Santo Antônio") \
                        .str.replace(".*(COTEMIG|Cotemig).*", "COTEMIG") \
                        .str.replace("(Ufam|.*Univ.*Fe.*Amaz.*|.*Fed.*Univer.*Amazonas.*)", "UFAM - Universidade Federal do Amazonas") \
                        .str.replace(".*UEA", "UEA - Universidade do Estado do Amazonas") \
                        .str.replace(".*Inst.*Fed.*cnologia.*Amazonas.*", "IFAM - Instituto Federal de Educação, Ciência e Tecnologia do Amazonas")\
                        .str.replace(".*Fund.*Nokia.*", "Fundação Nokia de Ensino") \
                        .str.replace(".*P.*U.*C.*Minas.*", "PUCMG - Pontifícia Universidade Católica de Minas Gerais") \
                        .str.replace("^P.*U.*C.*(SP|São Paulo)", "PUCSP - Pontifícia Universidade Católica de São Paulo") \
                        .str.replace(".*Universidade Paulista.*", "UNIP - Universidade Paulista") \
                        .str.replace(".*Estácio.*Sá.*", "Universidade Estácio de Sá") \
                        .str.replace(".*Univ.*Alfenas.*", "UNIFAL - Universidade Federal de Alfenas") \
                        .str.replace(".*F(ucapi|UCAPI)", "Instituto de Ensino Superior FUCAPI")




#formacao.loc[formacao.school.str.contains("UFPR", case=False)]
#formacao['perfil_id'].loc[formacao.school.str.contains("(Ufam|.*Univer.*Fed.*Amazonas.*|.*Fed.*Univer.*Amazonas.*)", case=False)]
#formacao.loc[formacao.school.str.contains("(Ufam|.*Univer.*Amazonas.*)", case=False)]
#print(formacao.instituicao.sort_values().unique())
#print(formacao.school.sort_values().unique())

#formacao.instituicao.sort_values().unique()

## 03.4 - Área de Formação

Agrupe as formações em grupos mais macros, por exemplo: Engenharia de Alimento, Engharia Elétrica estariam no grupo de engenharias, já Ciência da Computação, Sistemas de Informação poderiam estar no grupo de T.I. Crie áreas que façam sentido, agrupe os dados criando uma nova coluna no dataframe chamada area.

In [528]:

formacao.area = formacao.major.str.replace("Área de estudo\r\n", "") \
                .str.replace(".*Comp.*|.*Data.*|.*[s,S][y,i]s.*|.*[s,S]oft.*|.*[I,i]nf.*|.*Dig.*Imag.*|.*Des.*Intera.*|.*Int.*Des.*|.*Exatas.*", "TI") \
                .str.replace(".*Eng.*|.*Ingé.*", "Engenharia") \
                .str.replace(".*Mark.*|.*Pub.*|.*Comu.*|.*Jor.*|.*Rel.*Púb.*|.*Red.*", "Comunicação e Marketing") \
                .str.replace(".*Psi.*|.*Hum.*|.*Pess.*|.*Tour.*", "Pessoas") \
                .str.replace(".*Direi.*", "Direito") \
                .str.replace(".*Quali.*|.*Exch.*|.*Adm.*|.*Bus.*|.*Fina.*|.*Org.*|.*Neg.*|.*Inn.*|.*Proj.*|.*Conta.*|.*Gest.*(Com|Emp|Prod).*|.*[E,e]con.*|.*Atua.*", "Gestão") \
                .str.replace(".*Desi.*|.*Fine.*|.*Culi.*|.*Cine.*", "Design e Artes") \
                .str.replace(".*High*|\-", "Outros")

formacao.loc[formacao.major.str.contains("Miguel", case=False) 
             & (formacao.degree.str.contains("MARKET", case=False)), "area"] = "Comunicação e Marketing"

formacao.loc[formacao.major.str.contains("Miguel", case=False) 
             & (formacao.degree.str.contains("Sist", case=False)), "area"] = "TI"

formacao.loc[formacao.major.str.contains("-", case=False) 
             & (formacao.degree.str.contains("Publi|Mark|Comu", case=False)), "area"] = "Comunicação e Marketing"

formacao.loc[formacao.major.str.contains("-", case=False) 
             & (formacao.degree.str.contains("Sis|Infor|Banc|Help", case=False)), "area"] = "TI"

formacao.loc[formacao.major.str.contains("-", case=False) 
             & (formacao.degree.str.contains("Engi|Enge.*(Comp|Prod)", case=False)), "area"] = "Engenharia"

formacao.loc[formacao.major.str.contains("-", case=False) 
             & (formacao.degree.str.contains("Econ|Bach.*Adm|Emp|Entrep|Nego*", case=False)), "area"] = "Gestão"

formacao.loc[formacao.major.str.contains("-", case=False) 
             & (formacao.degree.str.contains("Psi", case=False)), "area"] = "Pessoas"


#formacao[['major', 'area', 'degree','instituicao']].loc[formacao.area.str.contains("Miguel", case=False) & (formacao.degree.str.contains("Help", case=False))]
#formacao.loc[formacao.major.str.contains("(.*Ci[ê,e].*Comp.*|.*Sist.*)", case=False)]
#formacao.area.sort_values().unique()
#formacao[['major', 'area', 'degree','instituicao']].loc[formacao.area.str.contains(".*High*|.*Hist.*|\-", case=False)]


C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


## 03.5 - Escolaridade

Uma informação consolidada de cada funcionário que pode ser derivada dessa base é a sua escolaridade (**escolaridade**), que é pode ser obtida pegando o maior grau daquele perfil. Gere uma base à parte contendo essa informação.

In [533]:
formacao[['perfil_id','date','grau','desc_grau','major','school']].sort_values(['grau','desc_grau'])

,perfil_id,date,grau,desc_grau,major,school
21,11,2002 – 2012,1,Ensino Médio,NaN,Colégio Santo Agostinho
38,20,2001 – 2014,1,Ensino Primário ao Ensino Médio,NaN,Colégio Santo Agostinho
43,22,2003 – 2005,1,Ensino médio,NaN,Escola Estadual Presidente Tancredo Neves
189,95,2006 – 2008,1,High School,Área de estudo\r\nHigh School,CSA | Colégio Santo Antônio
90,46,2014 – 2015,1,Non-degree Student,Área de estudo\r\nComputer Science,Kent State University
119,60,2000 – 2005,1,Under Graduate,Área de estudo\r\nFine Arts,Universidade Federal de Minas Gerais
153,77,2014 – 2014,2,Analise Google Analytics,Área de estudo\r\nMarketing,Gawa
80,41,2015 – 2015,2,Curso tecnico,Área de estudo\r\nBusiness e Marketing,Bridge Business College
164,83,2010 – 2012,2,Ensino Médio Integrado ao Técnico,Área de estudo\r\nInformática,Fundacao Nokia de Ensino
111,56,2011 – 2013,2,Ensino Médio e Técnico em Eletrônica,NaN,Colégio Técnico Industrial - Prof. Isaac Portal Roldán


## 03.6 Salvando Base Tratada

A base tratada será gravada no arquivo **formacao-tratado.csv**.

A base consolidadada com uma informação por registro será gravada no arquivo **perfil-formacao.csv** contendo as colunas: **perfil_id**, **escolaridade**, **escolaridade_numero**, **area**, **duracao**.

In [ ]:
# Grava a formação tratada em arquivo
formacao_tratado.to_csv('bases/formacao-tratado.csv')

# Grava o consolidade da formação por perfil em arquivo
formacao_perfil = pd.concat([escolaridade, area], axis=1)
formacao_perfil.to_csv('bases/perfil-formacao.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)
* [Continuar para o próximo documento](04-tratamento-base-habilidades.ipynb)